In [ ]:
# Baseline 1: ARMA model, NVDA spot price
# Baseline 2: ARMA model, log scale, NVDA spot price

ARMA model for NVDA spot price:
Scrape using Beautiful Soup, display using Pandas

In [34]:
# Imports
# To import, run "conda install -c conda-forge <package1> <package2>" in the Anaconda prompt
import os
import lxml
import pandas as pd
from bs4 import BeautifulSoup
import yfinance as yf
import datetime as dt
from datetime import date
import matplotlib.pyplot as plt
import numpy as np
import quantstats as qs


In [48]:
# Tickers
nvda = yf.Ticker("NVDA") 
qqq = yf.Ticker("QQQ")

In [49]:
data = nvda.history(period="5y", interval="1wk")
qqq_data = qqq.history(period="5y", interval="1wk")

In [53]:
print(data.head())

                               Open      High  ...  Dividends  Stock Splits
Date                                           ...                         
2020-02-24 00:00:00-05:00  6.524150  6.857503  ...      0.004           0.0
2020-03-02 00:00:00-05:00  6.897723  7.096758  ...      0.000           0.0
2020-03-09 00:00:00-04:00  5.976035  6.503640  ...      0.000           0.0
2020-03-16 00:00:00-04:00  5.291243  5.737390  ...      0.000           0.0
2020-03-23 00:00:00-04:00  5.124343  6.554457  ...      0.000           0.0

[5 rows x 7 columns]


In [51]:
#export
csvx = data[['Open', 'High', 'Low', 'Close']].to_csv("nvda_weekly_spot.csv", float_format="%.3f")
csvnq = qqq_data[['Open', 'High', 'Low', 'Close']].to_csv("qqq_weekly_spot.csv", float_format="%.3f")

In [52]:
print(data.describe())

             Open        High  ...   Dividends  Stock Splits
count  262.000000  262.000000  ...  262.000000    262.000000
mean    42.125947   44.504179  ...    0.000374      0.053435
std     41.121486   43.313976  ...    0.001432      0.664514
min      5.124343    5.737390  ...    0.000000      0.000000
25%     14.171155   14.829985  ...    0.000000      0.000000
50%     22.236909   23.060505  ...    0.000000      0.000000
75%     48.096403   49.811496  ...    0.000000      0.000000
max    148.669750  153.130005  ...    0.010000     10.000000

[8 rows x 7 columns]


In [42]:
data.info()
data.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 262 entries, 2020-02-24 00:00:00-05:00 to 2025-02-24 00:00:00-05:00
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          262 non-null    float64
 1   High          262 non-null    float64
 2   Low           262 non-null    float64
 3   Close         262 non-null    float64
 4   Volume        262 non-null    int64  
 5   Dividends     262 non-null    float64
 6   Stock Splits  262 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 16.4 KB


Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
dtype: int64

In [43]:
dfm = pd.read_csv("nvda_weekly_spot.csv")
print(dfm.head())

                        Date   Open   High    Low  Close
0  2020-02-24 00:00:00-05:00  6.524  6.858  6.020  6.724
1  2020-03-02 00:00:00-05:00  6.898  7.097  6.427  6.627
2  2020-03-09 00:00:00-04:00  5.976  6.504  5.376  5.999
3  2020-03-16 00:00:00-04:00  5.291  5.737  4.501  5.125
4  2020-03-23 00:00:00-04:00  5.124  6.554  4.945  6.296
